In [70]:
import requests
import csv
import os
import pandas as pd
import json
import re
from datetime import datetime
today_string = datetime.now().strftime('%Y-%m-%d')

In [71]:
input_path = "."

## Export Csv Personendaten

In [72]:
input_file = f"WIAG-Domherren-DB-Lebensdaten_2024-06-04.csv"
input_path_file = os.path.join(input_path, input_file)
persons_df = pd.read_csv(input_path_file, sep=';')
print(len(persons_df))
persons_df.head()

15606


,id,corpus,givenname,prefix,familyname,displayname,date_of_birth,date_of_death,biographical_dates,summary_offices,GND_ID,GSN,FactGrid_ID,Wikidata_ID,Wikipedia
0,WIAG-Pers-EPISCGatz-03848-001,epc,Ulrich,NaN,NaN,Ulrich,NaN,973,+ 973,"Bischof Augsburg 923-973, Domherr Domstift Aug...",118625284,059-01621-001,Q653546,Q714866,Ulrich_von_Augsburg
1,WIAG-Pers-EPISCGatz-03858-001,epc,Siegfried,NaN,NaN,Siegfried,NaN,1096,+ 1096,"Ernannter Bischof Augsburg 1077-1096, Domherr ...",13805746X,050-06892-001,Q653421,Q64763639,NaN
2,WIAG-Pers-EPISCGatz-04027-001,epc,Norbert,NaN,NaN,Norbert,NaN,NaN,~ 1079/1080,Bischof Chur 1079/1080-1088,NaN,710-00332-001,Q653192,Q35136089,Norbert_von_Hohenwart
3,WIAG-Pers-CANON-91597-001,dreg-can,Sizo,NaN,NaN,Sizo,NaN,NaN,~ um 1099,Domkustos Domstift Augsburg um 1099,NaN,710-00420-001,Q700910,NaN,NaN
4,WIAG-Pers-EPISCGatz-02518-001,epc,Udalschalk,NaN,NaN,Udalschalk,NaN,1202,+ 1202,"Bischof Augsburg 1184-1202, Dompropst Domstift...",138018383,059-01616-001,Q649639,Q68911884,NaN


In [73]:
url = 'https://database.factgrid.de/sparql'
query = """
SELECT ?person ?wiag WHERE {
  ?person wdt:P601 ?wiag.
  ?person wdt:P2 wd:Q7.
}
"""

r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
wiag_ids_df = pd.json_normalize(data['results']['bindings'])

len(wiag_ids_df)

11720

In [74]:
# extract out q id
def extract_qid(df, column):
    df[column] = df[column].map(lambda x: x.strip('https://database.factgrid.de/entity/'))
 
#factgrid_df['item.value'] = factgrid_df['item.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

# drop irrelevant columns
def drop_type_columns(df):
    df.drop(columns=[column for column in df.columns if column.endswith('type')], inplace=True)
    df.drop(columns=[column for column in df.columns if column.endswith('xml:lang')], inplace=True)

In [75]:
drop_type_columns(wiag_ids_df)
extract_qid(wiag_ids_df, 'person.value')
wiag_ids_df.columns = ['FactGrid_ID', 'old_wiag_id']
wiag_ids_df

,FactGrid_ID,old_wiag_id
0,Q653011,WIAG-Pers-EPISCGatz-05003-001
1,Q653013,WIAG-Pers-EPISCGatz-21313-001
2,Q653015,WIAG-Pers-EPISCGatz-04937-001
3,Q653016,WIAG-Pers-EPISCGatz-21140-001
4,Q653017,WIAG-Pers-EPISCGatz-04905-001
...,...,...
11715,Q728943,WIAG-Pers-CANON-25314-001
11716,Q728944,WIAG-Pers-CANON-25321-001
11717,Q728945,WIAG-Pers-CANON-25323-001
11718,Q728946,WIAG-Pers-CANON-25324-001


In [76]:
merged_df = wiag_ids_df.merge(persons_df[['FactGrid_ID', 'id']], on='FactGrid_ID')
merged_df

,FactGrid_ID,old_wiag_id,id
0,Q653060,WIAG-Pers-EPISCGatz-04423-001,WIAG-Pers-EPISCGatz-04423-001
1,Q653061,WIAG-Pers-EPISCGatz-05246-001,WIAG-Pers-EPISCGatz-05246-001
2,Q653073,WIAG-Pers-EPISCGatz-03060-001,WIAG-Pers-EPISCGatz-03060-001
3,Q653838,WIAG-Pers-EPISCGatz-10036-001,WIAG-Pers-EPISCGatz-10036-001
4,Q653846,WIAG-Pers-EPISCGatz-05394-001,WIAG-Pers-EPISCGatz-05394-001
...,...,...,...
7632,Q728943,WIAG-Pers-CANON-25314-001,WIAG-Pers-CANON-25314-001
7633,Q728944,WIAG-Pers-CANON-25321-001,WIAG-Pers-CANON-25321-001
7634,Q728945,WIAG-Pers-CANON-25323-001,WIAG-Pers-CANON-25323-001
7635,Q728946,WIAG-Pers-CANON-25324-001,WIAG-Pers-CANON-25324-001


In [77]:
diff_df = merged_df[merged_df['old_wiag_id'] != merged_df['id']]
diff_df

,FactGrid_ID,old_wiag_id,id
7110,Q883790,WIAG-Pers-CANON-15151-001,WIAG-Pers-CANON-85540-001


In [78]:
fg_qs_csv = diff_df.copy()
fg_qs_csv.columns = ['qid', '-P601', 'P601']
fg_qs_csv['-P601'] = '"' + fg_qs_csv['-P601'] + '"'
fg_qs_csv['P601'] = '"' + fg_qs_csv['P601'] + '"'
fg_qs_csv.set_index('qid')
fg_qs_csv

,qid,-P601,P601
7110,Q883790,"""WIAG-Pers-CANON-15151-001""","""WIAG-Pers-CANON-85540-001"""


In [79]:
today_string = datetime.now().strftime('%Y-%m-%d')

In [80]:
fg_qs_csv.to_csv(f'factgrid_wiag_id_update_{today_string}.csv', index=False)

## Do a cyclic check for all entries in factgrid

In [81]:
wiag_ids_df

,FactGrid_ID,old_wiag_id
0,Q653011,WIAG-Pers-EPISCGatz-05003-001
1,Q653013,WIAG-Pers-EPISCGatz-21313-001
2,Q653015,WIAG-Pers-EPISCGatz-04937-001
3,Q653016,WIAG-Pers-EPISCGatz-21140-001
4,Q653017,WIAG-Pers-EPISCGatz-04905-001
...,...,...
11715,Q728943,WIAG-Pers-CANON-25314-001
11716,Q728944,WIAG-Pers-CANON-25321-001
11717,Q728945,WIAG-Pers-CANON-25323-001
11718,Q728946,WIAG-Pers-CANON-25324-001


In [68]:
for _, (fg_id, old_wiag_id) in wiag_ids_df.iterrows():
    r = requests.get(f'https://wiag-vocab.adw-goe.de/id/{old_wiag_id}?format=Json')
    data = r.json()
    try:
        wiag_qid = data['persons'][0]['identifier']['Factgrid'].split('/')[-1]
        if wiag_qid != fg_id:
            print(fg_id, old_wiag_id, wiag_qid)
    except Exception as e:
        print(data)

{'dioceses': [{'wiagId': 'WIAG-Inst-DIOCGatz-001-001', 'URI': 'https://wiag-vocab.adw-goe.de/id/WIAG-Inst-DIOCGatz-001-001', 'name': 'Augsburg', 'status': 'Bistum', 'dateOfFounding': 'Spätantike, bis 539 Metropolitanverband Mailand', 'dateOfDissolution': '1802/1803', 'altLabels': [{'altLabel': 'ecclesia Augustana', 'lang': 'la'}, {'altLabel': 'Bistum Augsburg', 'lang': 'de'}, {'altLabel': 'Augustensis diocesis', 'lang': 'la'}, {'altLabel': 'August. dioc.', 'lang': 'la'}, {'altLabel': 'Bishopric of Augsburg', 'lang': 'en'}, {'altLabel': 'Diocèse de Augsbourg', 'lang': 'fr'}, {'altLabel': 'Bisdom Augsburg', 'lang': 'nl'}], 'ecclesiasticalProvince': 'Mainz', 'bishopricSeat': 'Augsburg', 'identifier': {'GND': '4003616-9', 'VIAF': '153058031', 'Wikidata': 'Q837110', 'Wikipedia': 'https://de.wikipedia.org/wiki/Bistum_Augsburg'}, 'identifiersComment': 'Alle Normdaten nehmen sowohl auf das Fürstbistum als auch auf das heutige Bistum Augsburg Bezug.', 'references': [{'citation': 'Erwin Gatz (Hg

KeyboardInterrupt: 

In [62]:
for _, (fg_id, old_wiag_id) in wiag_ids_df.iterrows():
    print(fg_id, old_wiag_id)
    break

Q16188 WIAG-Pers-EPISCGatz-10475-001
